<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/fast_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import os
import shutil
  
!git clone https://www.github.com/keras-team/keras-contrib.git \
  && cd keras-contrib \
  && pip install git+https://www.github.com/keras-team/keras-contrib.git \
  && python convert_to_tf_keras.py \
  && USE_TF_KERAS=1 python setup.py install

!pip install scipy==1.1.0

Cloning into 'keras-contrib'...
remote: Enumerating objects: 3634, done.
remote: Total 3634 (delta 0), reused 0 (delta 0), pack-reused 3634
Receiving objects: 100% (3634/3634), 848.39 KiB | 16.01 MiB/s, done.
Resolving deltas: 100% (2332/2332), done.
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-epf0w189
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-epf0w189
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=7f039d0caab8439b56d7df9830c937ddfafcd0887ad8daeb56152bac926426d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-lr1ks5o4/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Changed imports in 73 files.
Those files were found in the directory /content/keras-contrib
running install
running bdist_egg
running egg_info
creating tf_keras_contrib.egg-info
writing tf_keras_contrib.egg-info/PKG-INFO

In [119]:
import scipy
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam
from keras.models import load_model, save_model

import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import numpy as np
import os
from glob import glob
from PIL import Image

import tensorflow as tf
from tensorflow.python.keras.backend import set_session, clear_session
tf.compat.v1.disable_v2_behavior()

from tqdm import tqdm

import cv2

from keras.applications.vgg19 import preprocess_input, VGG19

from keras.losses import mean_squared_error

In [4]:
# source: https://www.machinecurve.com/index.php/2020/02/10/using-constant-padding-reflection-padding-and-replication-padding-with-keras/
from keras.layers import Layer

'''
  2D Reflection Padding
  Attributes:
    - padding: (padding_width, padding_height) tuple
'''
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

In [187]:
def conv_layer(x, num_filters, kernel_size, stride):
  reflection_size = kernel_size // 2
  reflection_padding = (reflection_size, reflection_size)
  x = ReflectionPadding2D(reflection_padding)(x)
  x = Conv2D(filters=num_filters, kernel_size=kernel_size, strides=stride)(x)
  return x

In [203]:
def residual_block(x, num_filters):
  input = x
  x = conv_layer(x, num_filters, 3, 1)
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)

  x = conv_layer(x, num_filters, 3, 1)
  x = InstanceNormalization()(x)

  x = Concatenate(axis=-1)([input, x])
  return x

In [213]:
def deconv_layer(x, num_filters, kernel_size, stride, upsample_size):
  reflection_size = kernel_size // 2
  reflection_padding = (reflection_size, reflection_size)
  x = UpSampling2D(size=upsample_size)(x)
  x = ReflectionPadding2D(reflection_padding)(x)
  x = Conv2D(filters=num_filters, kernel_size=kernel_size, strides=stride)(x)
  return x

In [214]:
def style_net(input):
  x = conv_layer(input, 32, 9, 1) #conv1
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 64, 3, 2) # conv2
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 128, 3, 2) # conv3
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = residual_block(x, 128) # res1
  x = residual_block(x, 128) # res2
  x = residual_block(x, 128) # res3
  x = residual_block(x, 128) # res4
  x = residual_block(x, 128) # res5
  x = deconv_layer(x, 64, 3, 1, 2) # deconv1
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = deconv_layer(x, 32, 3, 1, 2) # deconv2
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 3, 9, 1) # deconv3? apparently.

  model = Model(input=input, output=x)
  return model

def style_model(net):
  input_img = Input(shape=(400, 400, 3))
  gen_img = net(input_img)
  gen_features = extractor(gen_img)
  
  model = Model(input=input_img, output=gen_features)

  return model

In [215]:
input = Input(shape=(400, 400, 3))
net = style_net(input)
optimizer = Adam()
model = style_model(net)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`


In [216]:
net.summary()

Model: "model_51"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_60 (InputLayer)           (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
reflection_padding2d_401 (Refle (None, 408, 408, 3)  0           input_60[0][0]                   
__________________________________________________________________________________________________
conv2d_404 (Conv2D)             (None, 400, 400, 32) 7808        reflection_padding2d_401[0][0]   
__________________________________________________________________________________________________
instance_normalization_376 (Ins (None, 400, 400, 32) 2           conv2d_404[0][0]                 
___________________________________________________________________________________________

In [217]:
project_path = '/content/drive/My Drive/hairy_gan/'
style_img_path = os.path.join(project_path, 'style.jpg')
input_img_path = os.path.join(project_path, 'content.jpg')

In [218]:
def load_image(img_path, img_type='normal'):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (400, 400))
  img = tf.image.convert_image_dtype(img, tf.float32)
  return img


In [219]:
input_img = load_image(input_img_path)
input_img = tf.expand_dims(input_img, axis=0)
style_img = load_image(style_img_path)
style_img = tf.expand_dims(style_img, axis=0)

In [220]:
vgg = VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

In [221]:
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1','block5_conv1']
content_layers = ['block4_conv2']

num_style_layers = len(style_layers)
num_content_layers = len(content_layers)

In [222]:
def feature_extractor(layer_names, model):
  outputs = [model.get_layer(name).output for name in layer_names]
  model = Model(inputs=[vgg.input], outputs=outputs)
  return model

In [223]:
extractor = feature_extractor(style_layers + content_layers, vgg)

In [224]:
def preprocess_imgs(imgs):
  imgs = imgs * 255.
  imgs = preprocess_input(imgs)
  return imgs

In [225]:
def gram_matrix(tensor):
  temp = tensor
  temp = tf.squeeze(temp)
  height, width, channels = temp.shape
  fun = tf.reshape(temp, [channels, height * width])
  result = tf.matmul(fun, fun, transpose_a=True)
  gram = tf.expand_dims(result, axis=0)
  return gram

In [280]:
def get_style_loss(gen_style, target_style):
  gen_style = gram_matrix(gen_style)
  return tf.keras.losses.MeanAbsoluteError()(target_style, gen_style)

def get_content_loss(gen_content, target_content):
  return tf.keras.losses.MeanAbsoluteError()(target_content, gen_content)

def get_total_loss(gen_features, target_features):
  target_style_features = target_features[:num_style_layers]
  target_content_features = target_features[num_style_layers:]

  gen_style_features = gen_features[:num_style_layers]  
  total_style_loss = sum([get_style_loss(gen_feature, target_feature) for gen_feature, target_feature in zip(gen_style_features, target_style_features)])

  gen_content_features = gen_features[num_style_layers:]
  total_content_loss = sum([get_content_loss(gen_feature, target_feature) for gen_feature, target_feature in zip(gen_content_features, target_content_features)])

  total_loss = style_weight * total_style_loss + content_weight * total_content_loss

  return total_loss, total_style_loss, total_content_loss

In [276]:
processed_style_img = preprocess_imgs(style_img)
style_features = extractor(processed_style_img)[:num_style_layers]
gram_style_features = [gram_matrix(feature) for feature in style_features]

In [282]:
@tf.function()
def train_step(input_img):
  with tf.GradientTape() as tape: # auto calculates gradinets
    outputs = model(input_img)
    print(outputs)
    loss, _, _ = get_total_loss(outputs, input_features)

    grad = tape.gradient(loss, model)

    opt.apply_gradients([(grad, model)]) # wait.. we are applying the gradient to the image not the model..???

    image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)) # clip pixels to be in range of [0, 1]

In [278]:
epochs = 100
steps_per_epoch = 1
content_weight = 10
style_weight = 100
processed_input_img = preprocess_imgs(input_img)
input_features = extractor(processed_input_img)
input_content_features = input_features[num_style_layers:]
input_features = gram_style_features + input_content_features

In [283]:
for epoch in range(epochs):
  for i in tqdm(range(steps_per_epoch), desc=f'Train {epoch} / {epochs}', total=steps_per_epoch):
    train_step(processed_input_img)




Train 0 / 100:   0%|          | 0/1 [00:01<?, ?it/s]

[<tf.Tensor 'model_52/model_10/block1_conv1/Relu:0' shape=(1, 400, 400, 64) dtype=float32>, <tf.Tensor 'model_52/model_10/block2_conv1/Relu:0' shape=(1, 200, 200, 128) dtype=float32>, <tf.Tensor 'model_52/model_10/block3_conv1/Relu:0' shape=(1, 100, 100, 256) dtype=float32>, <tf.Tensor 'model_52/model_10/block4_conv1/Relu:0' shape=(1, 50, 50, 512) dtype=float32>, <tf.Tensor 'model_52/model_10/block5_conv1/Relu:0' shape=(1, 25, 25, 512) dtype=float32>, <tf.Tensor 'model_52/model_10/block4_conv2/Relu:0' shape=(1, 50, 50, 512) dtype=float32>]


AttributeError: ignored

AttributeError: ignored